In [9]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key from the environment variable
import mlflow
import os

In [10]:
from sklearn.pipeline import make_pipeline

In [11]:
os.environ["AWS_PROFILE"] = "default"
AWS_access_key_id = os.environ.get('aws_access_key_id')
AWS_secret_access_key = os.environ.get('aws_secret_access_key')

In [12]:


mlflow.set_tracking_uri("http://127.0.0.1:5000",)
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://s3-bucket-mlops/1', creation_time=1687333930628, experiment_id='1', last_update_time=1687333930628, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [13]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [14]:
!pwd

/mnt/f5a0bfae-56ba-4b82-a4ae-a4c94b49e822/mlops-zoomcamp-forked/04-deployment/web-service-mlflow


In [15]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [16]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [17]:
from mlflow.tracking import MlflowClient


In [18]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '125e1f20f7a544738d959329a37c9ecb'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [26]:


path = mlflow.artifacts.download_artifacts(run_id=RUN_ID)

In [27]:
path

'/tmp/tmpmlmhg7kb/'

In [28]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

IsADirectoryError: [Errno 21] Is a directory: '/tmp/tmpmlmhg7kb/'

In [23]:
dv

DictVectorizer()